### Logistic regression models of $C_t$ given $X$.


In [23]:
from experiments.simulation import forward_simulation
from sklearn.linear_model import LogisticRegression
from hmm.types import FloatArray

import numpy as np

In [69]:
def simulated_c_and_x_data(
    n: int,
    t: int,
    amount_of_data: int
) -> tuple[list[list[int]], list[FloatArray]]:
    # Datas.
    cs, _, xs = forward_simulation(n, t)

    for _ in range(1, amount_of_data):
        c, _, x = forward_simulation(n, t)

        cs += c
        xs = np.vstack([xs, x])

    return cs, xs

In [ ]:
def fit_logistic_regression_model(n: int, t: int, amount_of_data: int, print_stuff: bool = True, **kwargs) -> tuple[LogisticRegression, float]:
    c, x = simulated_c_and_x_data(
        n=n,
        t=t,
        amount_of_data=amount_of_data
    )

    clf = LogisticRegression(**kwargs).fit(x, c)
    accuracy = clf.score(x, c)

    if print_stuff:
        print(f"Training accuracy of model (t={t}, n={n}, amount_of_data={amount_of_data}):", accuracy)

    return clf, accuracy

In [71]:
model_zoo = {
    time_steps: {}
}

In [74]:
clf.score(x, c)

0.955993